In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense ,Dropout , BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy
#importing libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from seaborn import heatmap
from tensorflow import keras
from tensorflow.keras.layers import Conv1D,MaxPool1D,BatchNormalization,Bidirectional,LSTM,Flatten,Input,Dense
from keras import Model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report , confusion_matrix
from tensorflow.keras.utils import to_categorical
import warnings

# Load training and testing CSV files


#Imbalanced found so rectifying the imbalance
smote = SMOTE (sampling_strategy='all',random_state=42)
x =train_df.drop (columns =187)
y = train_df[187]
x,y = smote.fit_resample (x,y)
train_df = pd.concat([x,y],axis = 1)
# randomize dataset to prevent bias if any 
train_df = train_df.sample (frac =1) 

X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values
X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

# Integer encode the target labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Reshape input data for 1D convolutional layer
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Build TCN model
num_classes = len(np.unique(y_train))
model = Sequential([
    Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=X_train.shape[1:]),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Define early stopping callback based on validation loss
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fit the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Convert integer labels to one-hot encoded labels
y_test_onehot = to_categorical(y_test)

# Get true and predicted class labels from model predictions
y_true_classes = np.argmax(y_test_onehot, axis=1)  # Convert one-hot encoded labels to class labels
y_pred_classes = np.argmax(model.predict(X_test), axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d", xticklabels=np.unique(y_true_classes), yticklabels=np.unique(y_true_classes))
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()



In [ ]:
# Save the model
model.save("tcn_model.h5")


In [4]:
import pandas as pd

# Load the CSV dataset into a DataFrame
df = pd.read_csv("/kaggle/input/heartbeat/mitbih_train.csv")

# Get the unique class labels present in the dataset
unique_classes = df.iloc[:, -1].unique()

# Dictionary to store the index of one data point for each class
one_data_point_index_per_class = {}

# Loop through each unique class label
for class_label in unique_classes:
    # Find the index of the first occurrence of the class label
    data_point_index = df[df.iloc[:, -1] == class_label].index[0]
    # Store the index of the selected data point for this class label
    one_data_point_index_per_class[class_label] = data_point_index

# Print the dictionary containing the index of one data point for each class
print("Index of one data point for each class:")
print(one_data_point_index_per_class)


Index of one data point for each class:
{0.0: 0, 1.0: 72470, 2.0: 74693, 3.0: 80481, 4.0: 81122}


In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd

# Load the saved model
loaded_model = load_model("D:/College/Major Project/tcn_model.h5")
train_df = pd.read_csv("mitbih_train.csv", header=None)
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values

# Example: Selecting one data point from the dataset
data_point_index = 0  # Adjust this index to select the desired data point
data_point = X_train[0]  # Assuming X_test is your dataset
data_point = np.expand_dims(data_point, axis=0)  # Add batch dimension

# Make prediction on the data point
prediction = loaded_model.predict(data_point)

# Print the predicted class probabilities
print("Predicted probabilities:", prediction)

# Get the predicted class (assuming it's the class with highest probability)
predicted_class = np.argmax(prediction)
print("Predicted class:", predicted_class)


TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 187, 1], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']